## Prepare Model

In [1]:
import tensorflow as tf

In [2]:
class TanhModel(tf.keras.Model):
    
    def call(self, input_tensor):
        return tf.keras.activations.tanh(input_tensor)

In [3]:
model = TanhModel()

In [4]:
import numpy as np

In [5]:
def representive_dataset(num_data=1024):
    def dataset():
        for _ in range(num_data):
            yield [np.random.rand(128).astype('float32')]
    return dataset

In [6]:
model._set_inputs(np.random.rand(128).astype('float32'))

In [7]:
convert = tf.lite.TFLiteConverter.from_keras_model(model)

In [8]:
convert.representative_dataset = representive_dataset()

In [9]:
convert.optimizations = [tf.lite.Optimize.DEFAULT]
# convert.optimizations = []

In [10]:
tflm_model = convert.convert()

In [11]:
with open('tanh_model.tflite', 'wb') as fid:
    fid.write(tflm_model)

## Evaluation

In [12]:
interpretor = tf.lite.Interpreter(model_content=tflm_model)

In [13]:
interpretor.allocate_tensors()

In [14]:
x = np.random.rand(128)

In [15]:
in_idx = interpretor.get_input_details()[0]['index']

In [16]:
interpretor.set_tensor(in_idx, x.astype('float32'))

In [17]:
interpretor._interpreter.NumTensors()

4

In [18]:
interpretor.get_tensor_details()

[{'name': 'input_1_int8',
  'index': 0,
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.003921540454030037, -128),
  'quantization_parameters': {'scales': array([0.00392154], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'Identity_int8',
  'index': 1,
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.0078125, 0),
  'quantization_parameters': {'scales': array([0.0078125], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'input_1',
  'index': 2,
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': 

In [25]:
interpretor._interpreter.NodeName(1)

'TANH'

In [26]:
interpretor._interpreter.NodeInputs(1)

array([0], dtype=int32)

In [27]:
interpretor._interpreter.NodeOutputs(1)

array([1], dtype=int32)

In [29]:
interpretor._interpreter.NumNodes()

3

In [31]:
interpretor._interpreter.NodeName(1)

'TANH'

In [28]:
interpretor.get_tensor_details()

[{'name': 'input_1_int8',
  'index': 0,
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.003921540454030037, -128),
  'quantization_parameters': {'scales': array([0.00392154], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'Identity_int8',
  'index': 1,
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.0078125, 0),
  'quantization_parameters': {'scales': array([0.0078125], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'input_1',
  'index': 2,
  'shape': array([128], dtype=int32),
  'shape_signature': array([128], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': 

In [19]:
interpretor.invoke()

In [20]:
out_idx = interpretor.get_output_details()[0]['index']

In [21]:
out = interpretor.tensor(out_idx)

In [22]:
out()

array([0.5234375, 0.25     , 0.75     , 0.671875 , 0.703125 , 0.3125   ,
       0.5625   , 0.5      , 0.3203125, 0.7265625, 0.125    , 0.4375   ,
       0.609375 , 0.65625  , 0.6796875, 0.515625 , 0.3125   , 0.1484375,
       0.03125  , 0.296875 , 0.0078125, 0.03125  , 0.609375 , 0.109375 ,
       0.4140625, 0.28125  , 0.0625   , 0.4765625, 0.1484375, 0.3984375,
       0.3671875, 0.734375 , 0.2890625, 0.4140625, 0.5625   , 0.734375 ,
       0.7265625, 0.       , 0.3671875, 0.0390625, 0.25     , 0.3515625,
       0.1171875, 0.1328125, 0.4375   , 0.4609375, 0.328125 , 0.71875  ,
       0.3515625, 0.2734375, 0.0546875, 0.5234375, 0.7109375, 0.7578125,
       0.5      , 0.4609375, 0.3359375, 0.703125 , 0.703125 , 0.6953125,
       0.109375 , 0.140625 , 0.6875   , 0.625    , 0.671875 , 0.296875 ,
       0.6875   , 0.4296875, 0.296875 , 0.734375 , 0.75     , 0.3828125,
       0.4296875, 0.71875  , 0.7421875, 0.0390625, 0.5625   , 0.671875 ,
       0.1875   , 0.671875 , 0.4453125, 0.3359375, 

In [23]:
np.isclose(np.tanh(x), out(), atol=0.01).all()

True